# Auhors: Andrea ZANON, Hamza ZERHOUNI

In [1]:
using DataFrames
using JuMP, Gurobi

Create first basic model

In [15]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7; # we suppose week starts on tuesday, beacause monday is actually part of the previous football week

# define mondays, tuesdays, wednesdays, thursdays, fridays, saturdays, sundays. Suppose we start playing on friday
#=
fr = [7*k + 1 for k in 0:18]
sa = [7*k + 2 for k in 0:18]
su = [7*k + 3 for k in 0:18]
mo = [7*k + 4 for k in 0:18]
tu = [7*k + 5 for k in 0:18]
we = [7*k + 6 for k in 0:18]
th = [7*k + 7 for k in 0:18]
;
=#

# list with elements of fr, sa, su, mo


We define the matrix $C \in R^{TxT}$, with $C_{ij} = \frac{1}{j-i}$

In [38]:
C = zeros((W*(D+1), W*(D+1)))

for t in 1:W*(D+1)-1
    for t_ in t+1:W*(D+1)
        C[t,t_] = 1/(t_ - t)
    end
end

Simple model, we are considering single league and making assumption that each team pla

In [39]:
model = Model(Gurobi.Optimizer)

# define variables
# 1 if team i plays team j on day d of week w, 0 otherwise
@variable(model, x[i = 1:N, j = 1:N, w = 1:W, d = 1:D], Bin)

# define objective
# remember week begins on tuesday, this is why w_ goes from w+1
@objective(model, Min, sum(C[7*w + d, 7 * w_ + d_]* x[i, j, w, d] * x[i, k, w_, d_] for i in 1:N, j = 1:N, k = 1:N, w = 1:W, w_ = w+1:W, d = 1:D, d_ = 1:D))

# define constraints
# each team can never play itself
@constraint(model, [i = 1:N, w = 1:W, d = 1:D], sum(x[i, i, w, d]) == 0)

# each team plays at most one team per week
@constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w, d] for j = 1:N, d = 1:D) <= 1)

# if team A plays team B, then team B plays team A
@constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

# each team plays exactly N-1 games
@constraint(model, [i = 1:N], sum(x[i, j, w, d] for j = 1:N, w = 1:W, d = 1:D) == N-1)

# all teams need to play each other exactly once
@constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w = 1:W, d = 1:D) == 1)

# at least four days of rest (if they play monday, they can't play again until saturday)
@constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w+1, d1] for j = 1:N, d1 = 1:D) - sum(x[i, k, w, d2] for k = 1:N, d2 = 1:D) >= 4)

# can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
@constraint(model, [i = 1:N, j = 1:N, w = 1:W], sum(x[i, j, w, d] for d = 1:3) == 0)

# solve model
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


LoadError: InterruptException: